Amadéo David (44761700) - Marine Van Renterghem (31621700) - Sylvie Van Schendel (44841700)

### I.4 Résolution numérique du problème I.3 avec les données fournies

In [6]:
from scipy.optimize import linprog # import du solveur d'optimisation linéaire à utiliser
import matplotlib.pyplot as plt # pour graphiques éventuels
import numpy as np

In [7]:
def readFile(file_name):
    """
    Cette fonction lit un fichier formatté au type utilise dans les exemples du
    projet
    @pre                : Le fichier file_name est organise de la meme maniere
                          que les exemples donnes sur Moodle.
    
    @param file_name    : contient un String avec le nom du fichier a lire.
    
    @return n_points    : Contient un 'int' avec le nombre de points
    @return n_conduites : Contient un 'int' avec le nombre de conduites
    @return X           : Contient un 'numpy.array' avec les abscisses des points.
    @return Y           : Contient un 'numpy.array' avec les ordonnees des points.
    @return Z           : Contient un 'numpy.array' avec la hauteur des points.
    @return A           : Contient un 'numpy.array' avec la matrice d'incidence
                          des conduites.
    @return alpha       : Contient un 'float' avecla valeur de alpha.
    @return R           : Contient un 'float' avec le rayon des conduites.
    @return pa          : Contient un 'numpy.array' avec les points
                          d'approvisionnement.
    @return da_max      : Contient un 'numpy.array' avec les debits maximaux
                          extractibles.
    @return C           : Contient un 'numpy.array' avec les couts d'extraction.
    @return pc          : Contient un 'numpy.array' avec les points de
                          consommation.
    @return dc_min      : Contient un 'numpy.array' avec les debits minimaux
                          consommables.
    @return dc_max      : Contient un 'numpy.array' avec les debits maximaux
                          consommables.
    @return Pf          : Contient un 'float' avec le prix facture.
    @return pi          : Contient un 'numpy.array' avec les points
                          intermediaires.
    @return unit        : Contient un 'String' avec l'unite des coordonnes des
                          points.
    """
    with open(file_name,'r') as f:
        n_points    = (int)(f.readline().split()[0])
        n_conduites = (int)(f.readline().split()[0])
        f.readline()
        unit        = (f.readline().split(" = ")[1])
        
        coord       = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(n_points)))
        X           = coord[:,0]
        Y           = coord[:,1]
        Z           = coord[:,2]
        
        f.readline()
        f.readline()
        A           = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(n_points)))
        f.readline()
        
        UL_alpha    = f.readline().split(" = ")[1]
        UL_alpha    = UL_alpha.split()[0]
        UL2_alpha   = np.array(UL_alpha.split("^"))
        if(len(UL2_alpha) == 1):
            alpha   = (float)(UL2_alpha)
        else:
            alpha   = (float)(UL2_alpha[0]) ** (float)(UL2_alpha[1])
        
        R           = (float)(f.readline().split()[4])
        f.readline()
        
        UL_pa       = f.readline().split(" = [")[1]
        UL_pa       = UL_pa.split("]")[0]
        pa          = np.array(list(int(w) for w in UL_pa.split()[:]))
        
        UL_da_max   = f.readline().split(" = [")[1]
        UL_da_max   = UL_da_max.split("]")[0]
        da_max      = np.array(list(float(w) for w in UL_da_max.split()[:]))
        
        UL_C        = f.readline().split(" = [")[1]
        UL_C        = UL_C.split("]")[0]
        C           = np.array(list(float(w) for w in UL_C.split()[:]))
        
        f.readline()
        
        UL_pc       = f.readline().split(" = [")[1]
        UL_pc       = UL_pc.split("]")[0]
        pc          = np.array(list(int(w) for w in UL_pc.split()[:]))
        
        UL_dc_min   = f.readline().split(" = [")[1]
        UL_dc_min   = UL_dc_min.split("]")[0]
        dc_min      = np.array(list(float(w) for w in UL_dc_min.split()[:]))
        
        UL_dc_max   = f.readline().split(" = [")[1]
        UL_dc_max   = UL_dc_max.split("]")[0]
        dc_max      = np.array(list(float(w) for w in UL_dc_max.split()[:]))
        
        Pf          = (float)(f.readline().split()[3])
        
        f.readline()
        
        UL_pi       = f.readline().split("[")[1]
        UL_pi       = UL_pi.split("]")[0]
        pi          = np.array(list(float(w) for w in UL_pi.split()[:]))
        
    return (n_points, n_conduites, X, Y, Z, A, alpha, R, pa, da_max, C, pc, dc_min, dc_max, Pf, pi, unit)

In [58]:
#Fonction retournant [f,xC,xA] avec f la valeur de la fonction objectif maximale 
#xC un vecteur des valeurs optimales des débits aux points de consommation
#xA un vecteur des valeurs optimales des débits aux points d'approvisionnement
#Soit Pf le prix facturé
#Soit C le cout
#Soit DCmin et DCmax les valeurs minimales et maximales du débit aux points de consommation
#Soit DAmax les valeurs maximales du débit extractibles aux sources
def Analyse(nPoints,nConduites,X,Y,Z,A, alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi, unit):
    L=np.zeros(nConduites)
    for i in range(nConduites):
        p=0
        P1=np.zeros(3)
        P2=np.zeros(3)
        while(A[p][i]==0): 
            p+=1
        P1[0]=X[p]
        P1[1]=Y[p]
        P1[2]=Z[p]
        for j in range(p+1,nPoints): 
            if A[j][i]==-1 or A[j][i]==1 : 
                P2[0]=X[j]
                P2[1]=Y[j]
                P2[2]=Z[j]
                L[i]+=np.sqrt((P2[0]-P1[0])**2+(P2[1]-P1[1])**2+(P2[2]-P1[2])**2)
                P1=P2.copy()
                P2=np.zeros(3)
                
    h = A.T@Z       # le denivele de chaque conduite entre le pt de depart et d'arrivee.
    h = -1*h        # Difference de hauteur positive.
    
    A1     = np.concatenate((A[np.array(Pc)-1], -1*A[np.array(Pa)-1]))   # Suppression des points intermediaires
    
    DebMax = np.multiply(np.outer(np.ones(len(A1)), alpha*R*R*h/L), A1)  # Debit max, juste sur les points pas 
                                                                         # intermediaires
                                                                         # Est-ce normal d'avoir un carré?
                                                                         # Chaque ligne vaut le débit max absorbé en un point
                
    print("This is DebMax")
    print(DebMax)
    
    c  = -1 * np.concatenate((Pf, -1*np.array(C)))       # On traite un problème de maximisation
    
    print("This is c")
    print(c)
    
    A2 = np.eye(len(Dcmin)+len(Damax))
    
    
    A3 = np.ones((1,len(Dcmin)+len(Damax)))
    A3[0,len(Dcmin):(len(Dcmin)+len(Damax))]*=-1         # Assez d'approvisionnement pour la conso
                                                         # Bug, est-ce qu'on est sûr des indices a multiplier a -1?
                                                         # Probleme : on devrait avoir (0, -1, 1)
                                                         # et on a (1 1 -1) ?!
                                                         # Le code ne fonctionne toujours pas avec (0, -1, 1) (mm valeur sortie)
    print("This is A3")
    print(A3)
    A  = np.concatenate((A2,A3))
    print("This is A")
    print(A)
    #b  = np.concatenate((np.concatenate((Dcmax,Damax)),np.concatenate((-1*np.array(Dcmin),[0]))))
    Contraintes = DebMax @ np.ones(len(DebMax[0]))      # Contient le debit max absorbe par les conduites
    b = np.concatenate((Contraintes, [0]))
    
    BoundsLeft  = np.reshape(np.concatenate((Dcmin,np.zeros(len(Damax)))),(-1,1))
    BoundsRight = np.reshape(np.concatenate((Dcmax,Damax)),(-1,1))
    Bounds      = np.concatenate((BoundsLeft,BoundsRight),axis=1)
    
    print("This is Bounds :")
    print(Bounds)
    
    print("This is shape(BoundsLeft)")
    print(np.shape(BoundsLeft))
    
    
    res= linprog(c, A_ub = A, b_ub = b, bounds=Bounds)
    
    
    
    return [-1*res.fun]

In [59]:
(nPoints,nConduites,X,Y,Z,A, alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi, unit)=readFile("OurReseau/Donnees.txt")
print(Analyse(nPoints,nConduites,X,Y,Z,A,alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf*np.ones(len(Dcmin)),Pi, unit))

This is DebMax
[[ 0.57735027  0.          0.        ]
 [ 0.          0.          0.89442719]
 [ 0.57735027  0.57735027 -0.        ]]
This is c
[-1.  -1.   0.2]
This is A3
[[ 1.  1. -1.]]
[[ 1.  1. -1.]]
This is A
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  1. -1.]]
This is Bounds :
[[ 10. 120.]
 [ 10. 120.]
 [  0. 250.]]
This is shape(BoundsLeft)
(3, 1)
[-18.845299461620748]


In [45]:
B = [1, 2, 3, 4, 5]
A = np.outer(np.ones(2),B)
print(B[2:3])

[3]
